# 언어 모델의 기초적 이해 2

### 지난 시간 복습

지난 시간에 우리는 텍스트 데이터를 분석해서, 각 단어 다음에 등장할 확률을 직접 세어서 구한 다음, 계산한 확률을 토대로 문장을 생성해 나갔습니다.

여기에서, 우리는 수많은 문제점을 체험할 수 있었습니다:

1. 우리가 가지고 있는 텍스트 데이터에 시작 단어가 존재하지 않는 경우, 문장을 생성하지 못합니다.
2. 문장을 생성하더라도, 문맥의 흐름을 파악하지 못하고 자연스럽지 않은 문장이 생성되기도 합니다.

### 언어 모델 토큰화 다시 복습하기

In [ ]:
from transformers import AutoTokenizer

sample_text = "선생님 너무 잘생겼어요!"
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

tokens = tokenizer.tokenize(sample_text)
print(tokens)

actual_tokens = tokenizer(sample_text, return_tensors="pt").input_ids
print(actual_tokens)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

['▁선생', '님', '▁너무', '▁잘', '생', '겼', '어', '요', '!']
tensor([[22662,  7177, 12371,  9443,  7777,  6883,  8006,  8084,   376]])


### 지난 시간의 코드

In [ ]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=1b0e9f6575a9656d63206957a5af9a780f27a78dac9aec425dd30170022df004
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [ ]:
import re
import random
from tqdm import tqdm
from collections import defaultdict, Counter
from transformers import AutoTokenizer

def split_text_to_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

def remove_text_from_start_end_marker(text, start_marker='(', end_marker=')'):
    return re.sub(r'\{}.*?\{}'.format(re.escape(start_marker), re.escape(end_marker)), '', text).strip()

def clean_text_data(text):
    print("Cleaning text data...")
    sentences = split_text_to_sentences(text)
    sentences = [i.lower() for i in sentences] # make sentence lower cased. e.g. "Hello World" -> "hello world"
    sentences = [remove_text_from_start_end_marker(i) for i in sentences] # remove parentheses and their content. e.g. "hello world (test)" -> "hello world"
    to_replace = ["!", ";", '\n', '</p>', '<a', 'id=', "href=", 'title=', 'class=', '</a>', '(', ')', '}', '{',
                  '</sup>', '<p>', '</b>', '<sup', '>', '<', '\\', '-']
    replace_with = ''
    cleaned_sentences = []
    for i in sentences:
        word_array = i.split()
        word_array_new = []
        for word in word_array:
            for to_replace_val in to_replace:
                word = word.replace(to_replace_val, replace_with)
            word_array_new.append(word)
        cleaned_sentence = ' '.join(word_array_new).strip()
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence) # Remove extra whitespaces
        cleaned_sentences.append(cleaned_sentence)
    print("Cleaning complete.")
    return cleaned_sentences

def compute_next_token_probabilities(sentences, given_token_text, tokenizer=None):
    if tokenizer is None:
        print("No tokenizer provided. Creating a new tokenizer.")
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)
    given_token_id = tokenizer.convert_tokens_to_ids(given_token_text)
    next_token_counts = defaultdict(Counter)
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1
    total_next = sum(next_token_counts[given_token_id].values())
    if total_next == 0:
        return {}
    probabilities = {
        tokenizer.convert_ids_to_tokens(token_id): count / total_next
        for token_id, count in next_token_counts[given_token_id].items()
    }
    return probabilities

def compute_next_token_counts(tokens):
    next_token_counts = defaultdict(Counter)
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1
    return next_token_counts

def prepare_token_data(sentences, tokenizer=None):
    if tokenizer is None:
        print("No tokenizer provided. Creating a new tokenizer.")
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = []
    for sentence in tqdm(sentences, desc="Tokenizing..."):
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)
    return tokens

def sample_next_token(next_counts):
    tokens, counts = zip(*next_counts.items())
    total = sum(counts)
    probabilities = [count / total for count in counts]
    return random.choices(tokens, weights=probabilities, k=1)[0]

def random_sample_generate_sentence(next_token_counts, start_token_text, tokenizer=None, max_length=20):
    if tokenizer is None:
        print("No tokenizer provided. Creating a new tokenizer.")
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    current_token_id = tokenizer.convert_tokens_to_ids(start_token_text.lower())
    generated_tokens = [current_token_id]
    print("Generating a sentence with random sampling...")
    for _ in tqdm(range(max_length)):
        next_counts = next_token_counts.get(current_token_id, None)
        if not next_counts:
            break  # No next token found
        next_token_id = sample_next_token(next_counts)
        generated_tokens.append(next_token_id)
        current_token_id = next_token_id
        token_text = tokenizer.convert_ids_to_tokens(current_token_id)
        if token_text in ['.', '!', '?', tokenizer.sep_token, tokenizer.pad_token]:
            break
    generated_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(generated_tokens))
    return generated_text


In [ ]:
# Example usage:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Breakfast")
text = p_wiki.text
sentences = clean_text_data(text)

start_token_text = 'breakfast'
tokens = prepare_token_data(sentences, tokenizer=tokenizer)
next_token_counts = compute_next_token_counts(tokens)
generated_sentence = random_sample_generate_sentence(next_token_counts, start_token_text)
print("Generated sentence (random sample):")
print(generated_sentence)

Cleaning text data...
Cleaning complete.


Tokenizing...: 100%|██████████| 146/146 [00:00<00:00, 5875.62it/s]

No tokenizer provided. Creating a new tokenizer.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating a sentence with random sampling...


  0%|          | 0/20 [00:00<?, ?it/s]

Generated sentence (random sample):
breakfast


### 더 많은 데이터!

간단한 해결책은, 더 많은 텍스트 데이터를 사용하는 것입니다! 데이터는 많을 수록, 실제 언어 모델과 비슷해집니다.


In [ ]:
!pip install mwparserfromhell
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [ ]:
from datasets import load_dataset

wikipedia_dataset = load_dataset('wikipedia', '20220301.en')
print(wikipedia_dataset)

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

train-00001-of-00041.parquet:   0%|          | 0.00/705M [00:00<?, ?B/s]

train-00002-of-00041.parquet:   0%|          | 0.00/558M [00:00<?, ?B/s]

train-00003-of-00041.parquet:   0%|          | 0.00/491M [00:00<?, ?B/s]

...

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})


In [ ]:
from tqdm import tqdm

num_total_data = 10000
wikipedia_text_combined = ''
for i, data in enumerate(tqdm(wikipedia_dataset['train'])):
    wikipedia_text_combined += data['text']
    if i > num_total_data:
        break

  0%|          | 3403/6458670 [07:29<237:00:18,  7.57it/s]


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
wikipedia_sentences = clean_text_data(wikipedia_text_combined)
tokens = prepare_token_data(wikipedia_sentences, tokenizer=tokenizer)
next_token_counts = compute_next_token_counts(tokens)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Cleaning text data...
Cleaning complete.


Tokenizing...: 100%|██████████| 523424/523424 [01:42<00:00, 5117.69it/s]


In [ ]:
start_token_text = 'water'
generated_sentence = random_sample_generate_sentence(
    next_token_counts, start_token_text,
    tokenizer=tokenizer
)
print("Generated sentence (random sample):")
print(generated_sentence)

Generating a sentence with random sampling...


 45%|████▌     | 9/20 [00:00<00:00, 225.62it/s]

Generated sentence (random sample):
water for very large amounts, georgia campaign.


### 학습으로 넘어가기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


# Prepare dataset
class NextTokenDataset(Dataset):
    def __init__(self, sentences, tokenizer):
        self.inputs = []
        self.targets = []
        for sentence in sentences:
            tokens = tokenizer.encode(sentence, add_special_tokens=True)
            for i in range(len(tokens) - 1):
                self.inputs.append(tokens[i])
                self.targets.append(tokens[i+1])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

# Define simple MLP model
class SimpleMLP(nn.Module):
    def __init__(self, vocab_size, hidden_dim=128):
        super(SimpleMLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        x = self.embedding(x)
        return self.mlp(x)

In [ ]:
# Hyperparameters
batch_size = 64
epochs = 10
learning_rate = 1e-3

# Dataset and DataLoader
dataset = NextTokenDataset(wikipedia_sentences, tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, Optimizer
model = SimpleMLP(vocab_size=tokenizer.vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for prev_token, next_token in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        logits = model(prev_token)
        loss = criterion(logits, next_token)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1:  26%|██▌       | 64119/245005 [5:10:52<19:11:37,  2.62it/s]

In [ ]:
# Inference example
def generate_sequence(model, tokenizer, start_token, max_len=10):
    model.eval()
    token = tokenizer.encode(start_token, add_special_tokens=False)[-1]
    output_tokens = [token]

    for _ in range(max_len):
        input_tensor = torch.tensor([token])
        with torch.no_grad():
            logits = model(input_tensor)
            next_token = torch.argmax(logits, dim=-1).item()
            output_tokens.append(next_token)
            token = next_token

    return tokenizer.decode(output_tokens)

# Example generation
print(generate_sequence(model, tokenizer, "The"))

### (선택) 문맥의 흐름을 파악하여 자연스러운 문장 만들기

In [ ]:
# Prepare dataset
class NextTokenDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_seq_len=32):
        self.inputs = []
        self.targets = []
        for sentence in sentences:
            tokens = tokenizer.encode(sentence, add_special_tokens=True)
            for i in range(1, len(tokens)):
                input_seq = tokens[:i]
                target = tokens[i]
                if len(input_seq) > max_seq_len:
                    input_seq = input_seq[-max_seq_len:]
                self.inputs.append(input_seq)
                self.targets.append(target)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_seq = self.inputs[idx]
        target = self.targets[idx]
        return torch.tensor(input_seq), torch.tensor(target)

def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
    targets = torch.stack(targets)
    return inputs, targets

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim=128, num_layers=1):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        output = output[:, -1, :]  # take the output of the last token
        return self.fc(output)

In [ ]:
# Hyperparameters
batch_size = 64
epochs = 10
learning_rate = 1e-3

# Dataset and DataLoader
dataset = NextTokenDataset(sentences, tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Model, Loss, Optimizer
lstm = LSTMModel(vocab_size=tokenizer.vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for input_seq, next_token in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        logits = lstm(input_seq)
        loss = criterion(logits, next_token)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

In [ ]:
# Inference example
def generate_sequence(model, tokenizer, start_text, max_len=10):
    model.eval()
    tokens = tokenizer.encode(start_text, add_special_tokens=False)
    tokens = torch.tensor(tokens).unsqueeze(0)

    for _ in range(max_len):
        with torch.no_grad():
            logits = model(tokens)
            next_token = torch.argmax(logits, dim=-1).item()
            tokens = torch.cat([tokens, torch.tensor([[next_token]])], dim=1)

    return tokenizer.decode(tokens.squeeze().tolist())

# Example generation
print(generate_sequence(lstm, tokenizer, "The"))

### 문제점 논의하기